# import packages

In [ ]:
#This data collection code is partially based on the code provided by @aumyfarhan 
#for the Social Media Computing Class CS591

In [2]:
import tweepy
import yaml
import json
import sqlite3
from datetime import datetime
import pandas as pd

# Read twitter Authentication Keys

In [2]:
# yaml file reader funtion
def read_yaml(file_path):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

# yaml config file path
file_path = "twitter_api_key_config.yaml"
# read from config file
api_credential = read_yaml(file_path)

# Create Twitter Authentication

In [3]:
# API authentication
auth = tweepy.OAuthHandler(api_credential["api_key"], \
                           api_credential["api_secret_token"])
auth.set_access_token(api_credential["access_token"], \
                      api_credential["access_token_secret"])
api = tweepy.API(auth, wait_on_rate_limit=True)

# Create the Database and Required Tables

In [91]:
# establish a database connection
conn = sqlite3.connect('goodbot_table.db')
cur = conn.cursor()
create_tweet_info_table = """CREATE TABLE tweet_info(tweet_id BIGINT PRIMARY KEY, \
                                          user_id BIGINT, \
                                          tweet_lang TEXT, \
                                          tweet_time TEXT, \
                                          source TEXT, \
                                          tweet_text TEXT,\
                                          quote_count TEXT, \
                                          reply_count INT, \
                                          retweet_count INT,\
                                          tweet_favorite_count INT, \
                                          hashtags TEXT, \
                                          short_urls TEXT, \
                                          expanded_urls TEXT, \
                                          user_mentions TEXT, \
                                          id_str TEXT, \
                                          truncated INT, \
                                          in_reply_to_status_id BIGINT, \
                                          in_reply_to_status_id_str TEXT, \
                                          in_reply_to_user_id BIGINT, \
                                          in_reply_to_user_id_str TEXT, \
                                          in_reply_to_screen_name TEXT, \
                                          coordinates TEXT, \
                                          place TEXT
                                          );"""

create_user_info_table =  """CREATE TABLE user_info(user_id BIGINT PRIMARY KEY, \
                                          user_screen_name TEXT, \
                                          user_name TEXT, \
                                          user_language TEXT, \
                                          location TEXT, \
                                          profile_url TEXT, \
                                          description TEXT, \
                                          protected TEXT, \
                                          verified TEXT, \
                                          created_at TEXT, \
                                          friends_count BIGINT, \
                                          followers_count BIGINT,\
                                          favorites_count BIGINT, \
                                          statuses_count BIGINT, \
                                          id_str TEXT, \
                                          url TEXT);"""

cur.execute(create_tweet_info_table)
cur.execute(create_user_info_table)
conn.close()

# check if a table exists in the database

In [78]:
conn = sqlite3.connect('goodbot_table.db')
cur = conn.cursor()

table_exist_query = ''' SELECT count(*) FROM sqlite_master WHERE type='table' AND name='tweet_info' '''
cur.execute(table_exist_query)
exist_result = cur.fetchone()

if exist_result[0]==1:
    print("tweet_info table exists.")
else:
    print("tweet_info table does not exist.")
conn.close()

tweet_info table exists.


# define a StreamListener Object

In [10]:
# override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, listen_time=60):
        super(MyStreamListener, self).__init__()
        self.counter = 0
        print("Initialized Tweepy StreamListener.")
        self.start_time = datetime.now()
        self.current_time = datetime.now()
        self.listen_time = listen_time
        self.unique_user_id_set = set([])
        # adding database connection code
        self.conn = sqlite3.connect('tbt_table.db')
        self.cur  = self.conn.cursor()
        
    def insert_data(self, data):
        tweet_object=json.loads(data) # convert "string-line" into json
        # check if json object has a key id. Otherwise continue to next.
        if 'id' in tweet_object.keys(): 
            
            ##### Stuff I changed ######
            
            # tweet object information
            tweet_id        = tweet_object['id']                                #14
            user_id         = tweet_object['user']['id']                        #
            tweet_lang      = tweet_object['lang']                              #
            tweet_time      = str(pd.to_datetime(tweet_object['created_at']))   # created_at duplicate
            source          = tweet_object['source']                            #
            tweet_text      = tweet_object['text']                              #

            # tweet numeric information
            quote_count = tweet_object['quote_count']                           #
            reply_count = tweet_object['reply_count']                           #
            retweet_count = tweet_object['retweet_count']                       #
            tweet_favorite_count = tweet_object['favorite_count']               #

            # meta-content information
            hashtags = [str(hashtag['text']) for hashtag in tweet_object['entities']['hashtags']]  #
            hashtags = ",".join(hashtags)
            short_urls = [str(url['url']) for url in tweet_object['entities']['urls']]             #
            short_urls = ",".join(short_urls)
            expanded_urls = []                                                                     #
            try:
                expanded_urls = [str(url['expanded_url']) for url in tweet_object['entities']['urls']]
            except:
                print('Error Message: No Expanded URL.')
            expanded_urls = ",".join(expanded_urls)

            # user interaction based informations    
            user_mentions = [str(user_mentions['id'])\
                        for user_mentions in tweet_object['entities']['user_mentions']]
            user_mentions = ",".join(user_mentions)                                                 #


            # New Parameters
            # used try/except to ensure that program does not fail 

            # id_str
            try:
                id_str = tweet_object['id_str'] 
            except:
                id_str = None

            # truncated 
            try:
                truncated = tweet_object['truncated'] 
            except:
                truncated = None

            # in_reply_to_status_id 
            try:
                in_reply_to_status_id = tweet_object['in_reply_to_status_id'] 
            except:
                in_reply_to_status_id = None

            # in_reply_to_status_id_str 
            try:
                in_reply_to_status_id_str = tweet_object['in_reply_to_status_id_str'] 
            except:
                in_reply_to_status_id_str = None

            # in_reply_to_user_id  
            try:
                in_reply_to_user_id  = tweet_object['in_reply_to_user_id'] 
            except:
                in_reply_to_user_id  = None

            # in_reply_to_user_id_str 
            try:
                in_reply_to_user_id_str = tweet_object['in_reply_to_user_id_str'] 
            except:
                in_reply_to_user_id_str = None
            
            # in_reply_to_screen_name 
            try:
                in_reply_to_screen_name = tweet_object['in_reply_to_screen_name'] 
            except:
                in_reply_to_screen_name = None

            # coordinates 
            try:
                coordinates = tweet_object['coordinates'] 
            except:
                coordinates = None

            # place 
            try:
                place = tweet_object['place'] 
            except:
                place = None


            tweet_info = (tweet_id, user_id, tweet_lang,\
                    tweet_time, source, tweet_text,\
                    quote_count, reply_count, retweet_count,\
                    tweet_favorite_count, hashtags, short_urls,\
                    expanded_urls, user_mentions, id_str, truncated,\
                    in_reply_to_status_id, in_reply_to_status_id_str,\
                    in_reply_to_user_id, in_reply_to_user_id_str,\
                    in_reply_to_screen_name, coordinates, place)
            self.cur.execute("INSERT INTO tweet_info \
            VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", tweet_info)

            ########## End of my changes #############
            
            # user profile information

            if user_id in self.unique_user_id_set:
                pass
            else:
                self.unique_user_id_set.add(user_id)
                
                user_screen_name      = tweet_object['user']['screen_name']
                user_name             = tweet_object['user']['name']
                user_language         = tweet_object['user']['lang']  
                location              = tweet_object['user']['location']
                profile_url           = tweet_object['user']['url']
                description           = tweet_object['user']['description']
                protected             = tweet_object['user']['protected']
                verified              = tweet_object['user']['verified']
                created_at            = str(pd.to_datetime(tweet_object['user']['created_at']))
                friends_count         = tweet_object['user']['friends_count']
                followers_count       = tweet_object['user']['followers_count']
                favorites_count       = tweet_object['user']['favourites_count']
                statuses_count        = tweet_object['user']['statuses_count']
                
                try:
                    id_str = tweet_object['user']['id_str'] 
                except Exception as e:
                    print('Exception: id_str')
                    print(e)
                    id_str = None
                    
                try:
                    url = tweet_object['user']['url'] 
                except Exception as e:
                    print('Exception: url')
                    print(e)
                    url = None
                    
                    
                
                    
            

                user_information = (user_id, user_screen_name, user_name,\
                       user_language, location, profile_url,\
                       description, protected, verified, created_at,\
                       friends_count, followers_count,\
                       favorites_count, statuses_count, id_str, url)
                self.cur.execute(" INSERT INTO user_info VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", user_information)
            
            self.conn.commit()
        
    def on_data(self, data):
        self.current_time = datetime.now()
        time_elapsed = (self.current_time - self.start_time).total_seconds()
        if time_elapsed < self.listen_time:
            try:
                self.counter += 1
                """
                Changing the code here.
                Previously, we saved to Text file.
                Now, we will pass this to the Database insertor method.
                """
                # -- self.output_file.write(str(data))
                print(f"Tweet Processed: {self.counter}\n")
                self.insert_data(str(data))
                
            except Exception as e:
                print(f"On data Exception:{e}.")
        else:
            print(f"Stream listen time period ended. Total listen time: {self.listen_time} seconds.\n\n")
            print(f"Total Tweet processed: {self.counter}")
            self.conn.close()
            return False

    # handling Errors
    def on_error(self, status_code):
        print(f"status_code: {status_code}")
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

# create a stream

In [14]:
myStreamListener = MyStreamListener(listen_time=90000)
myStream = tweepy.Stream(api.auth, myStreamListener)

Initialized Tweepy StreamListener.


# start the streamer

In [17]:
keywords = ['#tbt']







try:
    print("Stream Filter")
    myStream.filter(track=keywords)
    print("DONE")
except Exception as e:
    print(f"error in stream filter {e}")

Stream Filter
Tweet Processed: 888

Tweet Processed: 889

Tweet Processed: 890

Tweet Processed: 891

Tweet Processed: 892

Tweet Processed: 893

Tweet Processed: 894

Tweet Processed: 895

Tweet Processed: 896

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 897

Tweet Processed: 898

Tweet Processed: 899

Tweet Processed: 900

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 901

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 902

Tweet Processed: 903

Tweet Processed: 904

Tweet Processed: 905

Tweet Processed: 906

Tweet Processed: 907

Tweet Processed: 908

Tweet Processed: 909

Tweet Processed: 910

Tweet Processed: 911

Tweet Processed: 912

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 913

Tweet Processed: 914

Tweet Processed: 915

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 916

Tweet Processed: 917

Tweet Proce

Tweet Processed: 1171

Tweet Processed: 1172

Tweet Processed: 1173

Tweet Processed: 1174

Tweet Processed: 1175

Tweet Processed: 1176

Tweet Processed: 1177

Tweet Processed: 1178

Tweet Processed: 1179

Tweet Processed: 1180

Tweet Processed: 1181

Tweet Processed: 1182

Tweet Processed: 1183

Tweet Processed: 1184

Tweet Processed: 1185

Tweet Processed: 1186

Tweet Processed: 1187

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 1188

Tweet Processed: 1189

Tweet Processed: 1190

Tweet Processed: 1191

Tweet Processed: 1192

Tweet Processed: 1193

Tweet Processed: 1194

Tweet Processed: 1195

Tweet Processed: 1196

Tweet Processed: 1197

Tweet Processed: 1198

Tweet Processed: 1199

Tweet Processed: 1200

Tweet Processed: 1201

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 1202

Tweet Processed: 1203

Tweet Processed: 1204

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 1205

Tweet 

KeyboardInterrupt: 

# fetch the data into a pandas dataframe

In [12]:
conn = sqlite3.connect('Covid_table_compressed.db')
cur = conn.cursor()

cur.execute("SELECT * FROM tweet_info")
tweet_info_all_result = cur.fetchall()
tweet_info_column_names = [description[0] for description in cur.description]
tweet_info_dataframe = pd.DataFrame(tweet_info_all_result, \
                                   columns=tweet_info_column_names)
print(f"No of tweet stored: {len(tweet_info_all_result)}, {tweet_info_dataframe.shape[0]}\n")



cur.execute("SELECT * FROM user_info")
user_info_all_result = cur.fetchall()
user_info_column_names = [description[0] for description in cur.description]
user_info_dataframe = pd.DataFrame(user_info_all_result, \
                                   columns=user_info_column_names)
print(f"No of user info stored: {len(user_info_all_result)}, {user_info_dataframe.shape[0]}\n")

conn.close()

No of tweet stored: 34824, 34824

No of user info stored: 26767, 26767



In [5]:
tweet_info_dataframe.head()

,tweet_id,user_id,tweet_lang,tweet_time,source,tweet_text,quote_count,reply_count,retweet_count,tweet_favorite_count,...,user_mentions,id_str,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,coordinates,place
0,1459610441450409984,551700157,in,2021-11-13 19:53:42+00:00,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",Tipe ideal Nana adalah Lee Sun Gyun\nNana deka...,0,0,0,0,...,,1459610441450409984,0,NaN,None,NaN,None,None,None,None
1,1459610446412259328,935681833,ja,2021-11-13 19:53:43+00:00,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",生産量1位はグンマー #bot,0,0,0,0,...,,1459610446412259328,0,NaN,None,NaN,None,None,None,None
2,1459610448920473600,2909597051,en,2021-11-13 19:53:43+00:00,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",Suggestions are always welcome! #BOT,0,0,0,0,...,,1459610448920473600,0,NaN,None,NaN,None,None,None,None
3,1459610461041983488,2880050479,ja,2021-11-13 19:53:46+00:00,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",にゃーん！ #bot,0,0,0,0,...,,1459610461041983488,0,NaN,None,NaN,None,None,None,None
4,1459610461792473093,1348512671486812160,vi,2021-11-13 19:53:46+00:00,"<a href=""https://mobile.twitter.com"" rel=""nofo...","RT @meopho270699: 😗😗😗 sáng sớm k có gì ăn, đượ...",0,0,0,0,...,1126017285787750401,1459610461792473093,0,NaN,None,NaN,None,None,None,None


# Sample one user from the dataset

In [73]:
sample = user_info_dataframe.sample(n=1)
sample['user_screen_name'].values

array(['NCPreps'], dtype=object)

# Add the bot rating column to the database table

In [83]:
conn = sqlite3.connect('tbt_table.db')
cur = conn.cursor()
add_bot_rating = """ALTER TABLE user_info \
                    ADD bot_rating DECIMAL(1,2);"""

cur.execute(add_bot_rating)
conn.close()

# Get a sample of 5, 10, 15, 25 or 50 users

In [70]:
sample = user_info_dataframe.sample(n=50)
sample['user_screen_name'].values

array(['madakisolarin', 'SmitheeRebecca', 'luanLeonardodi2',
       'Juan95347711', 'ay_greatness', 'JWill33fd', 'alfredodf63',
       '_WanSyafiq', 'D_LatinSuperFly', 'MelindaTolley', 'kuruzo03',
       'hagwood5', 'crispoliverrr', 'uwllibrarian', 'chema1223',
       'Domenico1oo777', 'ElpatronGusGus', 'ValidTripleA__', 'CodisGroup',
       'elportondemarte', 'akshay_tamizhan', 'kathuS30', 'realKPDinc',
       'DivyaDevaraj18', 'jeancm03', 'airwrecka_yo', 'johnste83319293',
       'kazanandrez', 'Thomas66880610', 'VitColucci_', 'Soccermama87',
       'SupportBalance', 'Quintus74689745', 'colombopana6', 'asianmoans',
       'Morenu_69', 'chelsiwonda', 'blackglitter27', 'jsquera',
       'ThelmaBartlet14', 'ilynewwie', 'helenaachagas', 'SoySashaMala',
       'DavidDDeaton2', 'lettistruluv', 'atlxpomx', 'FaceTheGod',
       'BBiitchhh', 'JoeGKushner', 'Call_me_Andre'], dtype=object)

# Update the bot rating in the database for an array of screen names

In [75]:
conn = sqlite3.connect('tbt_table.db')
cur = conn.cursor()

screen_names = ['madakisolarin', 'SmitheeRebecca', 'luanLeonardodi2',
       'Juan95347711', 'ay_greatness', 'JWill33fd', 'alfredodf63',
       '_WanSyafiq', 'D_LatinSuperFly', 'MelindaTolley', 'kuruzo03',
       'hagwood5', 'crispoliverrr', 'uwllibrarian', 'chema1223',
       'Domenico1oo777', 'ElpatronGusGus', 'kh_chopper_1551', 'CodisGroup',
       'elportondemarte', 'akshay_tamizhan', 'kathuS30', 'realKPDinc',
       'DivyaDevaraj18', 'jeancm03', 'airwrecka_yo', 'johnste83319293',
       'kazanandrez', 'Thomas66880610', 'VitColucci_', 'Soccermama87',
       'SupportBalance', 'Quintus74689745', 'colombopana6', 'asianmoans',
       'Morenu_69', 'chelsiwonda', 'blackglitter27', 'jsquera',
       'ThelmaBartlet14', 'windychicago123', 'NCPreps', 'SoySashaMala',
       'DavidDDeaton2', 'lettistruluv', 'atlxpomx', 'FaceTheGod',
       'BBiitchhh', 'JoeGKushner', 'Call_me_Andre']

scores = [0.4,1.5,0.4,4.6,1.7,1.8,0.4,
         4.0,0.8,0.3,0.6,1.1,0.4,1.6,
         3.6,1.3,0.6,0.8,4.4,0.6,3.1,
         2.0,0.3,3.2,3.8,0.8,4.6,2.7,
         4.2,0.3,0.0,1.0,0.5,4.4,3.8,
         3.4,4.1,1.8,0.0,4.1,4.7,1.0,
         0.0,1.4,3.2,3.8,0.2,4.8,1.2,
         0.8]

for num in range(0,50):
    update_bot_rating = """UPDATE user_info \
                           SET bot_rating ='""" + str(scores[num]) + """' \
                           WHERE user_screen_name='""" + screen_names[num] + """';"""
    
    print(update_bot_rating)

    cur.execute(update_bot_rating)
    
conn.commit()
conn.close()

UPDATE user_info                            SET bot_rating ='0.4'                            WHERE user_screen_name='madakisolarin';
UPDATE user_info                            SET bot_rating ='1.5'                            WHERE user_screen_name='SmitheeRebecca';
UPDATE user_info                            SET bot_rating ='0.4'                            WHERE user_screen_name='luanLeonardodi2';
UPDATE user_info                            SET bot_rating ='4.6'                            WHERE user_screen_name='Juan95347711';
UPDATE user_info                            SET bot_rating ='1.7'                            WHERE user_screen_name='ay_greatness';
UPDATE user_info                            SET bot_rating ='1.8'                            WHERE user_screen_name='JWill33fd';
UPDATE user_info                            SET bot_rating ='0.4'                            WHERE user_screen_name='alfredodf63';
UPDATE user_info                            SET bot_rating ='4.0'         

# Verify that the bot rating was updated. Query only the users with more than 3.5.

In [77]:
conn = sqlite3.connect('tbt_table.db')
cur = conn.cursor()

#cur.execute("SELECT * FROM user_info WHERE bot_rating > 3.5")

#Check how many users in total
cur.execute("SELECT * FROM user_info WHERE bot_rating IS NOT NULL")

user_info_all_result = cur.fetchall()
user_info_column_names = [description[0] for description in cur.description]
user_info_bot_dataframe = pd.DataFrame(user_info_all_result, \
                                   columns=user_info_column_names)
print(f"No of user info stored: {len(user_info_all_result)}, {user_info_dataframe.shape[0]}\n")

conn.close()

No of user info stored: 650, 38239



In [148]:
user_info_bot_dataframe.head(29)

,user_id,user_screen_name,user_name,user_language,location,profile_url,description,protected,verified,created_at,friends_count,followers_count,favorites_count,statuses_count,id_str,url,bot_rating
0,257044617,benhausdo,hausdo,None,None,https://instagram.com/hausdo,this is a funky introduction to how nice i am,0,0,2011-02-24 16:16:15+00:00,120,2039,8603,3843,257044617,https://instagram.com/hausdo,0.0
1,1197638608301244422,_mbarbaran,Maria Bárbara,None,"Minas Gerais, Brasil",None,simpática e amigável,0,0,2019-11-21 22:11:19+00:00,170,471,15443,6942,1197638608301244422,None,0.4
2,1234644626281127936,musicf00dtravel,_Tonya_,None,"New Mexico, USA",None,Music f00d travel,0,0,2020-03-03 01:00:45+00:00,122,156,11201,3742,1234644626281127936,None,0.6
3,2343195384,DLars06,Dallyboi,None,None,None,@chlo_shea,0,0,2014-02-14 07:03:49+00:00,402,476,21793,12874,2343195384,None,0.1
4,908377990056341504,niiteGodfrey,"Young, gifted and black",None,🇹🇿,None,"This is my life homie, you decide yours/\n\nMa...",0,0,2017-09-14 17:12:44+00:00,1182,1294,176055,3651,908377990056341504,None,0.4
5,3077892708,Lerato_Hloai,Rre Maoto🧐🇮🇹,None,SE 8,None,Aesthete\nSuper🌟star|\nBaller#2⚽|\nPsalm23|\n@...,0,0,2015-03-13 19:49:43+00:00,722,604,14447,4307,3077892708,None,1.6
6,1924204988,SurakhbayarG,Surakhbayar.G,None,"Ulaanbaatar, Mongolia",None,Reads #policy #mining #impactinvesting Interes...,0,0,2013-10-01 18:14:43+00:00,2303,3661,23536,42475,1924204988,None,3.6
7,1181213266032447490,ORodCinco,|-| O’neal ®©,None,None,http://nextgrouprd.com,🅲🅸🅶🅰🆁 🅵🅰🅽 | जीवन सरल छ | 🥑🥓☕️ | @NextGroupRD,0,0,2019-10-07 14:22:50+00:00,304,456,1026,12336,1181213266032447490,http://nextgrouprd.com,0.7
8,821568194070593536,ByAndyMcCulloch,Andrew McCulloch,None,"Houston, TX",https://muckrack.com/andrew-c-mcculloch/portfolio,Award-winning journalist covering Texas HS spo...,0,0,2017-01-18 04:01:36+00:00,4976,5494,48763,36482,821568194070593536,https://muckrack.com/andrew-c-mcculloch/portfolio,3.4
9,1157718971577225216,SexyColombiacom,SexyColombia.com +18 💋,None,None,None,None,0,0,2019-08-03 18:24:46+00:00,1597,7646,819,2058,1157718971577225216,None,3.8


# close the database connection

In [73]:
conn.close()